In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Saskylakh,RU,2021-12-04 01:58:45,71.9167,114.0833,-17.16,97,100,13.73
1,1,Albany,US,2021-12-04 01:58:45,42.6001,-73.9662,31.62,68,4,6.85
2,2,Hilo,US,2021-12-04 01:58:45,19.7297,-155.0900,79.79,84,100,12.21
3,3,Upernavik,GL,2021-12-04 01:58:46,72.7868,-56.1549,10.42,70,92,9.24
4,4,Goderich,CA,2021-12-04 01:58:46,43.7501,-81.7165,35.91,86,95,3.44


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
#heat_layer = gmaps.heatmap_layer(locations, weights=temps)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig 


Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Hilo,US,2021-12-04 01:58:45,19.7297,-155.0900,79.79,84,100,12.21
6,6,Atuona,PF,2021-12-04 01:58:47,-9.8000,-139.0333,78.93,71,1,16.33
11,11,Carnarvon,AU,2021-12-04 01:58:49,-24.8667,113.6333,77.07,69,0,12.66
14,14,Petatlan,MX,2021-12-04 01:58:50,17.5167,-101.2667,75.78,87,66,3.15
15,15,Rikitea,PF,2021-12-04 01:58:50,-23.1203,-134.9692,76.73,73,43,15.35
24,24,Koungou,YT,2021-12-04 01:58:54,-12.7336,45.2042,76.71,94,20,4.61
31,31,Takoradi,GH,2021-12-04 01:58:57,4.8845,-1.7554,79.57,85,81,7.74
34,34,Avarua,CK,2021-12-04 01:58:59,-21.2078,-159.7750,75.25,94,90,4.61
35,35,Saint-Philippe,RE,2021-12-04 01:58:59,-21.3585,55.7679,76.55,85,36,8.57
36,36,Samarai,PG,2021-12-04 01:59:00,-10.6167,150.6667,84.07,74,64,9.08


In [10]:
preferred_cities_df.count()

City_ID       130
City          130
Country       130
Date          130
Lat           130
Lng           130
Max Temp      130
Humidity      130
Cloudiness    130
Wind Speed    130
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Hilo,US,79.79,19.7297,-155.0900,
6,Atuona,PF,78.93,-9.8000,-139.0333,
11,Carnarvon,AU,77.07,-24.8667,113.6333,
14,Petatlan,MX,75.78,17.5167,-101.2667,
15,Rikitea,PF,76.73,-23.1203,-134.9692,
24,Koungou,YT,76.71,-12.7336,45.2042,
31,Takoradi,GH,79.57,4.8845,-1.7554,
34,Avarua,CK,75.25,-21.2078,-159.7750,
35,Saint-Philippe,RE,76.55,-21.3585,55.7679,
36,Samarai,PG,84.07,-10.6167,150.6667,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [15]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
      hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
      print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Hilo,US,79.79,19.7297,-155.0900,Hilo Hawaiian Hotel
6,Atuona,PF,78.93,-9.8000,-139.0333,Villa Enata
11,Carnarvon,AU,77.07,-24.8667,113.6333,Hospitality Carnarvon
14,Petatlan,MX,75.78,17.5167,-101.2667,Hotel California Petatlán
15,Rikitea,PF,76.73,-23.1203,-134.9692,People ThankYou
...,...,...,...,...,...,...
537,Barahona,DO,78.12,18.2085,-71.1008,Hotel Caribe
538,Progreso,MX,76.10,21.2833,-89.6667,Playa Linda Hotel
539,Tecoanapa,MX,78.55,16.5167,-98.7500,Hotel Manglares
543,Santa Rosa,PH,84.09,14.3122,121.1114,Erlyn’s Sta Rosa City Home


In [16]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [22]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))